In [1]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
# We use r"..." to handle Windows backslashes correctly
BASE_DIR = r"C:\Users\arjun\Downloads\Nutri Rate"
PARQUET_FILE = r"C:\Users\arjun\Downloads\food.parquet"
INDIAN_CSV = r"C:\Users\arjun\Downloads\Indian_Food_Nutrition_Processed.csv"
USDA_FOLDER = r"C:\Users\arjun\Downloads\FoodData_Central_csv_2025-12-18\FoodData_Central_csv_2025-12-18"

print("✅ Environment Ready. Paths set.")

✅ Environment Ready. Paths set.


In [2]:
def calculate_nutri_grade(row):
    """
    Calculates a simplified Nutri-Score (A-E) based on:
    - Points for Bad stuff (Sugar, Sat Fat, Calories)
    - Points for Good stuff (Protein, Fiber)
    """
    # 1. if Grade already exists, keep it
    if pd.notna(row.get('grade')) and row['grade'] in ['A', 'B', 'C', 'D', 'E']:
        return row['grade']

    # 2. If missing crucial data, return Unknown
    if pd.isna(row.get('sugar')) or pd.isna(row.get('fat')):
        return "UNKNOWN"

    # 3. Calculate "Bad Points" (N)
    points_n = 0
    # Energy
    if row['calories'] > 335: points_n += 4
    elif row['calories'] > 100: points_n += 2
    # Sugar
    if row['sugar'] > 45: points_n += 10
    elif row['sugar'] > 22: points_n += 6
    elif row['sugar'] > 4.5: points_n += 2
    # Sat Fat (Approximation)
    if row['fat'] > 10: points_n += 10
    elif row['fat'] > 4: points_n += 4

    # 4. Calculate "Good Points" (P)
    points_p = 0
    # Protein
    if row['protein'] > 8: points_p += 5
    elif row['protein'] > 1.6: points_p += 2

    # 5. Final Score & Mapping
    final_score = points_n - points_p

    if final_score <= -1: return 'A'    # Very Healthy
    if final_score <= 2: return 'B'     # Healthy
    if final_score <= 10: return 'C'    # Balanced
    if final_score <= 18: return 'D'    # Limit
    return 'E'                          # Unhealthy

print("✅ Grading Logic Defined.")

✅ Grading Logic Defined.


In [3]:
import pyarrow.parquet as pq

# Load the file metadata (this is instant)
parquet_file = pq.ParquetFile(PARQUET_FILE)

# Get all column names
all_columns = parquet_file.schema.names

print(f"Total Columns found: {len(all_columns)}")

# Check specifically for the ones we need
required_checks = [
    'code', 'product_name', 'countries_tags', 'nutriscore_grade',
    'energy-kcal_100g', 'energy_100g', 
    'sugars_100g', 'fat_100g', 'proteins_100g', 
    'image_url', 'image_small_url'
]

print("\n--- CHECKING REQUIRED COLUMNS ---")
found_cols = []
for col in required_checks:
    if col in all_columns:
        print(f"✅ Found: {col}")
        found_cols.append(col)
    else:
        print(f"❌ MISSING: {col}")

# Print first 20 columns to see the naming style
print("\n--- FIRST 20 COLUMNS (For Reference) ---")
print(all_columns[:20])

Total Columns found: 144

--- CHECKING REQUIRED COLUMNS ---
✅ Found: code
❌ MISSING: product_name
❌ MISSING: countries_tags
✅ Found: nutriscore_grade
❌ MISSING: energy-kcal_100g
❌ MISSING: energy_100g
❌ MISSING: sugars_100g
❌ MISSING: fat_100g
❌ MISSING: proteins_100g
❌ MISSING: image_url
❌ MISSING: image_small_url

--- FIRST 20 COLUMNS (For Reference) ---
['additives_n', 'element', 'element', 'element', 'brands', 'categories', 'element', 'ciqual_food_code', 'agribalyse_food_code', 'agribalyse_proxy_food_code', 'element', 'element', 'element', 'code', 'compared_to_category', 'complete', 'completeness', 'element', 'element', 'created_t']


In [4]:
import pyarrow.parquet as pq

# Load the file metadata again
parquet_file = pq.ParquetFile(PARQUET_FILE)
all_columns = parquet_file.schema.names

print(f"Total Columns: {len(all_columns)}")

# Define keywords to search for
search_terms = ['name', 'energy', 'sugar', 'fat', 'protein', 'image', 'url']

print("\n--- 🔍 COLUMN SEARCH RESULTS ---")

for term in search_terms:
    print(f"\n👉 Searching for '{term.upper()}':")
    matches = [c for c in all_columns if term in c.lower()]
    
    # Print the first 5 matches to avoid spamming
    if matches:
        for match in matches[:5]:
            print(f"   Found: {match}")
    else:
        print("   ❌ No match found.")

print("\n---------------------------------")

Total Columns: 144

--- 🔍 COLUMN SEARCH RESULTS ---

👉 Searching for 'NAME':
   Found: name
   Found: field_name

👉 Searching for 'ENERGY':
   ❌ No match found.

👉 Searching for 'SUGAR':
   ❌ No match found.

👉 Searching for 'FAT':
   ❌ No match found.

👉 Searching for 'PROTEIN':
   ❌ No match found.

👉 Searching for 'IMAGE':
   Found: last_image_t

👉 Searching for 'URL':
   ❌ No match found.

---------------------------------


In [5]:
import pandas as pd
import pyarrow.parquet as pq

# 1. Print ALL 144 Columns
parquet_file = pq.ParquetFile(PARQUET_FILE)
all_columns = parquet_file.schema.names

print(f"--- ALL {len(all_columns)} COLUMNS ---")
# Print in chunks of 10 so it's readable
for i in range(0, len(all_columns), 10):
    print(all_columns[i:i+10])

# 2. Inspect the Data (The "X-Ray")
# We will load just the first 3 rows of the ENTIRE file to see what's inside.
print("\n--- 🔍 INSPECTING FIRST ROW DATA ---")
df_sample = pd.read_parquet(PARQUET_FILE).head(1)

# We iterate through the columns of the first row to find numbers
for col in df_sample.columns:
    val = df_sample.iloc[0][col]
    # If the value is a dictionary or looks like nutrition data, print it
    if isinstance(val, dict) or (isinstance(val, str) and "energy" in str(val)):
        print(f"\n👉 Column '{col}' contains: {val}")
        
# Check specifically for a 'nutriments' column (standard for Open Food Facts)
if 'nutriments' in df_sample.columns:
    print("\n✅ FOUND 'nutriments' column! Content:")
    print(df_sample.iloc[0]['nutriments'])
else:
    print("\n❌ 'nutriments' column NOT found in top level.")
    

--- ALL 144 COLUMNS ---
['additives_n', 'element', 'element', 'element', 'brands', 'categories', 'element', 'ciqual_food_code', 'agribalyse_food_code', 'agribalyse_proxy_food_code']
['element', 'element', 'element', 'code', 'compared_to_category', 'complete', 'completeness', 'element', 'element', 'created_t']
['creator', 'element', 'element', 'element', 'element', 'ecoscore_data', 'ecoscore_grade', 'ecoscore_score', 'element', 'element']
['element', 'emb_codes', 'element', 'element', 'lang', 'text', 'key', 'imgid', 'rev', 'h']
['w', 'h', 'w', 'h', 'w', 'h', 'w', 'uploaded_t', 'uploader', 'element']
['element', 'ingredients_from_palm_oil_n', 'ingredients_n', 'element', 'ingredients_percent_analysis', 'element', 'lang', 'text', 'ingredients_with_specified_percent_n', 'ingredients_with_unspecified_percent_n']
['ingredients_without_ciqual_codes_n', 'element', 'ingredients', 'known_ingredients_n', 'element', 'labels', 'lang', 'element', 'element', 'last_editor']
['last_image_t', 'last_modif

In [7]:
import pyarrow.parquet as pq

print("🔍 SCANNING FILE SCHEMA (Safe Mode)...")

# 1. Get ALL available column names directly from the file
parquet_file = pq.ParquetFile(PARQUET_FILE)
all_columns = set(parquet_file.schema.names) # Use a set for fast lookup

print(f"File has {len(all_columns)} columns.")

# 2. Define the columns we WANT (and their aliases)
# We map 'Target Name' -> ['Possible Name 1', 'Possible Name 2']
wanted_columns = {
    'barcode': ['code', 'barcode', 'id'],
    'name': ['product_name', 'name', 'generic_name'],
    'brand': ['brands', 'brand', 'brand_owner'],
    'grade': ['nutriscore_grade', 'nutrition_grades', 'nutrition_grade_fr'],
    'source_info': ['manufacturing_places', 'origins', 'countries_tags'],
    'nutrition_blob': ['nutriments', 'nutrition_data', 'nutriments_estimated']
}

# 3. Find the ACTUAL names in this specific file
final_cols = []
mapping = {} # To rename them later

for target, possibilities in wanted_columns.items():
    found = False
    for candidate in possibilities:
        if candidate in all_columns:
            final_cols.append(candidate)
            mapping[candidate] = target # Store how to rename it
            print(f"   ✅ Found '{target}' as column: '{candidate}'")
            found = True
            break
    if not found:
        print(f"   ⚠️ WARNING: Could not find column for '{target}'")

# 4. Load ONLY the columns we found
print(f"\n⏳ Loading {len(final_cols)} columns from Parquet...")
try:
    df_off = pd.read_parquet(PARQUET_FILE, columns=final_cols)
    
    # 5. Rename to standard names
    df_off = df_off.rename(columns=mapping)
    print(f"Loaded {len(df_off)} rows.")

    # 6. Filter for INDIA
    # We check 'source_info' (was manufacturing_places) for 'India'
    # We also check 'brand' for common Indian brands
    print("Filtering for Indian products...")
    
    # Create a mask for filtering
    mask = pd.Series(False, index=df_off.index)
    
    if 'source_info' in df_off.columns:
        mask |= df_off['source_info'].str.contains('india', case=False, na=False)
    
    if 'brand' in df_off.columns:
        mask |= df_off['brand'].str.contains('Amul|Britannia|Parle|Tata|Haldiram|Dabur|Nestle India', case=False, na=False)
        
    df_india = df_off[mask].copy()
    print(f"Filtered down to {len(df_india)} Indian products.")

    # 7. Extract Nutrition (The Hard Part)
    # Only run this if we actually found the 'nutrition_blob' column
    if 'nutrition_blob' in df_india.columns:
        print("Extracting Sugar/Fat/Protein from nested data...")
        
        def extract_val(blob, key_name):
            # Safe extraction function
            if isinstance(blob, list) or isinstance(blob, np.ndarray):
                for item in blob:
                    if isinstance(item, dict) and item.get('name') == key_name:
                        # Return the value (prioritize 100g)
                        return float(item.get('100g', item.get('value', 0.0)))
            return 0.0

        # Create new columns
        df_india['sugar'] = df_india['nutrition_blob'].apply(lambda x: extract_val(x, 'sugars'))
        df_india['fat'] = df_india['nutrition_blob'].apply(lambda x: extract_val(x, 'fat'))
        df_india['protein'] = df_india['nutrition_blob'].apply(lambda x: extract_val(x, 'proteins'))
        df_india['calories'] = df_india['nutrition_blob'].apply(lambda x: extract_val(x, 'energy-kcal'))
        
        # Drop the heavy blob column
        df_india = df_india.drop(columns=['nutrition_blob'])
    else:
        print("❌ CRITICAL: No nutrition data found. Creating empty columns.")
        df_india['sugar'] = 0.0
        df_india['fat'] = 0.0
        df_india['protein'] = 0.0
        df_india['calories'] = 0.0

    # 8. Final Clean
    if 'grade' in df_india.columns:
        df_india['grade'] = df_india['grade'].str.upper()
    
    df_india['source'] = 'OpenFoodFacts'
    
    # Save
    output_path = os.path.join(PROJECT_DIR, "processed_off_india_safe.csv")
    df_india.to_csv(output_path, index=False)
    print(f"\n✅ SUCCESS! Saved to: {output_path}")
    display(df_india.head())

except Exception as e:
    print(f"\n❌ STILL CRASHING? Error: {e}")

🔍 SCANNING FILE SCHEMA (Safe Mode)...
File has 86 columns.
   ✅ Found 'barcode' as column: 'code'
   ✅ Found 'name' as column: 'name'
   ✅ Found 'brand' as column: 'brands'
   ✅ Found 'grade' as column: 'nutriscore_grade'
   ✅ Found 'source_info' as column: 'manufacturing_places'
   ⚠️ WARNING: Could not find column for 'nutrition_blob'

⏳ Loading 5 columns from Parquet...

❌ STILL CRASHING? Error: No match for FieldRef.Name(name) in additives_n: int32
additives_tags: list<element: string>
allergens_tags: list<element: string>
brands_tags: list<element: string>
brands: string
categories: string
categories_tags: list<element: string>
categories_properties: struct<ciqual_food_code: int32, agribalyse_food_code: int32, agribalyse_proxy_food_code: int32>
checkers_tags: list<element: string>
ciqual_food_name_tags: list<element: string>
cities_tags: list<element: string>
code: string
compared_to_category: string
complete: int32
completeness: float
correctors_tags: list<element: string>
countr

In [8]:
import pyarrow.parquet as pq
import pandas as pd
import os

print("🚀 STARTING BATCH PROCESSING (The Safe Way)...")

# 1. Open the Parquet file stream
parquet_file = pq.ParquetFile(PARQUET_FILE)
print(f"File found. Total Row Groups: {parquet_file.num_row_groups}")

# 2. Define lists to hold our Indian products
indian_products = []
batch_count = 0

# 3. Iterate through the file in batches (chunks)
# This prevents RAM crashes and bypasses "Column Missing" errors
try:
    for batch in parquet_file.iter_batches(batch_size=10000):
        batch_count += 1
        
        # Convert the batch to a Pandas DataFrame
        df_chunk = batch.to_pandas()
        
        # --- DYNAMIC COLUMN MAPPING (Finds columns automatically) ---
        # We look for ANY column that looks like 'country' or 'place'
        country_col = next((c for c in df_chunk.columns if 'countr' in c.lower() or 'place' in c.lower()), None)
        brand_col = next((c for c in df_chunk.columns if 'brand' in c.lower()), None)
        
        # If we can't find a country column, we can't filter safely, so we skip
        if not country_col and not brand_col:
            continue

        # --- FILTERING FOR INDIA ---
        mask = pd.Series(False, index=df_chunk.index)
        
        # Check Country/Place column
        if country_col:
            mask |= df_chunk[country_col].astype(str).str.contains('india', case=False, na=False)
            
        # Check Brand column (Backup filter)
        if brand_col:
            mask |= df_chunk[brand_col].astype(str).str.contains('Amul|Britannia|Parle|Tata|Haldiram|Nestle', case=False, na=False)
        
        # Apply filter
        df_india_chunk = df_chunk[mask].copy()
        
        # If we found Indian products in this chunk, save them
        if not df_india_chunk.empty:
            print(f"   Batch {batch_count}: Found {len(df_india_chunk)} Indian items.")
            indian_products.append(df_india_chunk)
            
        # Stop after checking 50 batches (approx 500k rows) just to test
        # Remove this break if you want to scan the WHOLE file (takes longer)
        if batch_count >= 50: 
            print("   (Stopping early for test run...)")
            break

    # 4. MERGE & SAVE
    if indian_products:
        print("\n✅ Merging found products...")
        final_df = pd.concat(indian_products, ignore_index=True)
        
        # NOW we select columns (Safety Check: only keep columns that actually exist)
        keep_cols = [c for c in final_df.columns if c in ['code', 'product_name', 'brands', 'nutriscore_grade', 'image_url']]
        final_df = final_df[keep_cols]
        
        # Rename for clarity
        rename_map = {
            'code': 'barcode',
            'product_name': 'name', 
            'brands': 'brand',
            'nutriscore_grade': 'grade',
            'image_url': 'image'
        }
        final_df = final_df.rename(columns=rename_map)
        
        # Add Source
        final_df['source'] = 'OpenFoodFacts'
        
        # Save
        output_path = os.path.join(PROJECT_DIR, "india_products_batch_v1.csv")
        final_df.to_csv(output_path, index=False)
        print(f"🎉 SUCCESS! Saved {len(final_df)} Indian products to: {output_path}")
        display(final_df.head())
        
    else:
        print("❌ No Indian products found in the first 50 batches.")

except Exception as e:
    print(f"❌ Error during batching: {e}")

🚀 STARTING BATCH PROCESSING (The Safe Way)...
File found. Total Row Groups: 4190
   Batch 1: Found 1 Indian items.
   Batch 2: Found 7 Indian items.
   Batch 3: Found 29 Indian items.
   Batch 6: Found 7 Indian items.
   Batch 7: Found 41 Indian items.
   Batch 8: Found 24 Indian items.
   Batch 9: Found 4 Indian items.
   Batch 10: Found 3 Indian items.
   Batch 11: Found 6 Indian items.
   Batch 12: Found 2 Indian items.
   Batch 13: Found 11 Indian items.
   Batch 14: Found 13 Indian items.
   Batch 15: Found 4 Indian items.
   Batch 16: Found 8 Indian items.
   Batch 17: Found 5 Indian items.
   Batch 18: Found 5 Indian items.
   Batch 19: Found 1 Indian items.
   Batch 20: Found 44 Indian items.
   Batch 21: Found 10 Indian items.
   Batch 22: Found 3 Indian items.
   Batch 23: Found 5 Indian items.
   Batch 24: Found 575 Indian items.
   Batch 25: Found 10 Indian items.
   Batch 26: Found 49 Indian items.
   Batch 27: Found 4 Indian items.
   Batch 28: Found 2 Indian items.
   Ba

In [10]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import os

# --- 1. SETUP PATHS ---
PROJECT_DIR = r"C:\Users\arjun\Downloads\Nutri Rate"
PARQUET_FILE = r"C:\Users\arjun\Downloads\food.parquet"

print("🚀 STARTING ROBUST FINAL PROCESSING...")

# --- 2. THE FIXED EXTRACTION FUNCTION ---
def extract_nutrient(nutriments_data, target_name):
    """
    Safely extracts value. Handles None, 'None', and missing keys.
    """
    # 1. Check if the input is a valid list/array
    if not isinstance(nutriments_data, (list, np.ndarray)):
        return 0.0

    # 2. Loop through the list
    for item in nutriments_data:
        if isinstance(item, dict) and item.get('name') == target_name:
            # 3. Try to get '100g' first, then 'value'
            val = item.get('100g')
            if val is None:
                val = item.get('value')
            
            # 4. If we still have None, return 0.0
            if val is None:
                return 0.0
                
            # 5. Try converting to float safely
            try:
                return float(val)
            except (ValueError, TypeError):
                return 0.0
                
    return 0.0

# --- 3. BATCH PROCESS ---
parquet_file = pq.ParquetFile(PARQUET_FILE)
indian_products = []
batch_count = 0

print(f"File found. Scanning in batches...")

# Iterate through file
for batch in parquet_file.iter_batches(batch_size=10000):
    batch_count += 1
    
    df_chunk = batch.to_pandas()
    
    # A. Dynamic Column Search
    country_col = next((c for c in df_chunk.columns if 'countr' in c.lower() or 'place' in c.lower()), None)
    brand_col = next((c for c in df_chunk.columns if 'brand' in c.lower()), None)
    # Search for 'nutriments' or 'nutrition_data' or 'nutrition_grades'
    nutri_col = next((c for c in df_chunk.columns if 'nutriment' in c.lower() or 'nutrition_data' in c.lower()), None)

    # B. Filter for India
    mask = pd.Series(False, index=df_chunk.index)
    if country_col:
        mask |= df_chunk[country_col].astype(str).str.contains('india', case=False, na=False)
    if brand_col:
        mask |= df_chunk[brand_col].astype(str).str.contains('Amul|Britannia|Parle|Tata|Haldiram|Nestle|Dabur', case=False, na=False)
    
    df_india_chunk = df_chunk[mask].copy()

    # C. Process Data
    if not df_india_chunk.empty:
        # Extract Nutrition using the FIXED function
        if nutri_col:
            # We use lambda to apply the function row by row
            df_india_chunk['sugar'] = df_india_chunk[nutri_col].apply(lambda x: extract_nutrient(x, 'sugars'))
            df_india_chunk['fat'] = df_india_chunk[nutri_col].apply(lambda x: extract_nutrient(x, 'fat'))
            df_india_chunk['protein'] = df_india_chunk[nutri_col].apply(lambda x: extract_nutrient(x, 'proteins'))
            df_india_chunk['calories'] = df_india_chunk[nutri_col].apply(lambda x: extract_nutrient(x, 'energy-kcal'))
        else:
            df_india_chunk['sugar'] = 0.0
            df_india_chunk['fat'] = 0.0
            df_india_chunk['protein'] = 0.0
            df_india_chunk['calories'] = 0.0

        print(f"   Batch {batch_count}: Found {len(df_india_chunk)} items.")
        indian_products.append(df_india_chunk)

    # Limit to 500 batches for a good sample (Remove this logic to scan EVERYTHING)
    if batch_count >= 500: 
        print("   (Stopping at 500 batches...)")
        break

# --- 4. MERGE & SAVE ---
if indian_products:
    print("\n✅ Merging datasets...")
    final_df = pd.concat(indian_products, ignore_index=True)
    
    # Clean Column Names
    rename_map = {
        'code': 'barcode', 
        'product_name': 'name', 
        'brands': 'brand', 
        'nutriscore_grade': 'grade',
        'image_url': 'image'
    }
    # Only rename columns that exist
    final_df = final_df.rename(columns=rename_map)
    
    # Keep only relevant columns
    cols_we_want = ['barcode', 'name', 'brand', 'grade', 'sugar', 'fat', 'protein', 'calories', 'image']
    final_cols = [c for c in cols_we_want if c in final_df.columns]
    
    final_df = final_df[final_cols]
    final_df['source'] = 'OpenFoodFacts'

    # Save
    output_path = os.path.join(PROJECT_DIR, "india_products_final.csv")
    final_df.to_csv(output_path, index=False)
    
    print(f"🎉 SUCCESS! Saved {len(final_df)} Indian products to:")
    print(f"📂 {output_path}")
    display(final_df.head())
else:
    print("❌ No Indian products found.")
    

🚀 STARTING ROBUST FINAL PROCESSING...
File found. Scanning in batches...
   Batch 1: Found 1 items.
   Batch 2: Found 7 items.
   Batch 3: Found 29 items.
   Batch 6: Found 7 items.
   Batch 7: Found 41 items.
   Batch 8: Found 24 items.
   Batch 9: Found 4 items.
   Batch 10: Found 3 items.
   Batch 11: Found 6 items.
   Batch 12: Found 2 items.
   Batch 13: Found 11 items.
   Batch 14: Found 13 items.
   Batch 15: Found 4 items.
   Batch 16: Found 8 items.
   Batch 17: Found 5 items.
   Batch 18: Found 5 items.
   Batch 19: Found 1 items.
   Batch 20: Found 44 items.
   Batch 21: Found 10 items.
   Batch 22: Found 3 items.
   Batch 23: Found 5 items.
   Batch 24: Found 575 items.
   Batch 25: Found 10 items.
   Batch 26: Found 49 items.
   Batch 27: Found 4 items.
   Batch 28: Found 2 items.
   Batch 30: Found 1 items.
   Batch 31: Found 9 items.
   Batch 32: Found 3 items.
   Batch 33: Found 2 items.
   Batch 34: Found 58 items.
   Batch 35: Found 12 items.
   Batch 36: Found 2 item

,barcode,name,brand,grade,sugar,fat,protein,calories,source
0,00024907,"[{'lang': 'main', 'text': 'Teriyaki Dip Sauce ...",Marks & Spencer,e,0.0,0.0,0.0,0.0,OpenFoodFacts
1,0012000713019,[],Nestlé,unknown,0.0,0.0,0.0,0.0,OpenFoodFacts
2,0012345678905,"[{'lang': 'main', 'text': 'Sphère plaisir'}, {...",Alteña's Nopales,unknown,0.0,0.0,0.0,0.0,OpenFoodFacts
3,0013800143310,"[{'lang': 'main', 'text': 'Italiano lasagna'},...",Nestlé,c,0.0,0.0,0.0,0.0,OpenFoodFacts
4,0015000046446,"[{'lang': 'main', 'text': 'Gerber Fruit & Yogu...",Nestlé,unknown,0.0,0.0,0.0,0.0,OpenFoodFacts


In [13]:
import ast

print("🧹 CLEANING DATA NAMES & VALUES...")

# 1. Load the file we just made (if not already in memory)
# final_df = pd.read_csv(os.path.join(PROJECT_DIR, "india_products_final.csv")) 

# 2. Function to fix the Name column
def fix_product_name(val):
    try:
        # If it's a list (already parsed), take the first item's text
        if isinstance(val, (list, np.ndarray)) and len(val) > 0:
             if isinstance(val[0], dict):
                return val[0].get('text', 'Unknown Product')
        
        # If it's a string looking like a list "[{'text':...}]"
        if isinstance(val, str) and val.startswith("["):
            # specific fix for the format seen in your output
            # We simple-parse it or regex it if ast fails, but let's try basic string slicing first
            # " ... 'text': 'REAL NAME' ... "
            import re
            match = re.search(r"'text':\s*'([^']*)'", val)
            if match:
                return match.group(1)
                
    except Exception:
        pass
    return val # Return original if we can't fix it

print("Fixing Names... (This is fast)")
final_df['name'] = final_df['name'].apply(fix_product_name)

# 3. Filter out "Ghost" Products (Where ALL nutrition is 0)
# If Calories + Sugar + Fat + Protein == 0, the data is likely missing.
# We keep them but mark them as "Needs Scan" or drop them. 
# Let's drop them to keep the database high-quality.
print(f"Original Count: {len(final_df)}")

# Filter: Keep rows where at least ONE nutrient is > 0
valid_nutrition = (final_df['calories'] > 0) | (final_df['sugar'] > 0) | (final_df['fat'] > 0) | (final_df['protein'] > 0)
clean_df = final_df[valid_nutrition].copy()

print(f"Cleaned Count: {len(clean_df)} (Removed {len(final_df) - len(clean_df)} empty rows)")

# 4. Save the Final Polish
clean_output = os.path.join(PROJECT_DIR, "india_products_clean.csv")
clean_df.to_csv(clean_output, index=False)

print(f"✨ DONE! Your Golden Database is at: {clean_output}")
display(clean_df.head())

🧹 CLEANING DATA NAMES & VALUES...
Fixing Names... (This is fast)
Original Count: 36616
Cleaned Count: 0 (Removed 36616 empty rows)
✨ DONE! Your Golden Database is at: C:\Users\arjun\Downloads\Nutri Rate\india_products_clean.csv


,barcode,name,brand,grade,sugar,fat,protein,calories,source


In [14]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import os
import ast

# --- 1. SETUP ---
PROJECT_DIR = r"C:\Users\arjun\Downloads\Nutri Rate"
PARQUET_FILE = r"C:\Users\arjun\Downloads\food.parquet"

print("🚀 RE-STARTING FINAL EXTRACTION (Universal Parser)...")

# --- 2. THE UNIVERSAL EXTRACTION FUNCTION ---
def universal_extractor(blob, target_name):
    """
    Handles List, String-of-List, or Struct.
    """
    # CASE A: It's a String (The most likely culprit)
    if isinstance(blob, str):
        try:
            # excessive safety: if it looks like a list, convert it
            if blob.strip().startswith('['):
                blob = ast.literal_eval(blob)
        except:
            return 0.0

    # CASE B: It's a List (Standard)
    if isinstance(blob, (list, np.ndarray)):
        for item in blob:
            if isinstance(item, dict) and item.get('name') == target_name:
                val = item.get('100g')
                if val is None: val = item.get('value')
                
                try: return float(val)
                except: return 0.0
    return 0.0

def clean_name(val):
    """ Fixes the [{'text': 'Name'}] issue immediately """
    if isinstance(val, (list, np.ndarray)) and len(val) > 0:
        if isinstance(val[0], dict): return val[0].get('text', 'Unknown')
    return val

# --- 3. BATCH PROCESS ---
parquet_file = pq.ParquetFile(PARQUET_FILE)
indian_products = []
batch_count = 0

for batch in parquet_file.iter_batches(batch_size=10000):
    batch_count += 1
    df_chunk = batch.to_pandas()
    
    # A. Find Columns
    # We prioritize 'nutriments' exactly
    nutri_col = 'nutriments' if 'nutriments' in df_chunk.columns else None
    
    # Fallback search if exact name missing
    if not nutri_col:
        nutri_col = next((c for c in df_chunk.columns if 'nutriment' in c.lower()), None)

    # B. Filter India
    mask = pd.Series(False, index=df_chunk.index)
    
    # Check manufacturing places / country
    place_col = next((c for c in df_chunk.columns if 'manufacturing' in c.lower() or 'country' in c.lower()), None)
    if place_col:
        mask |= df_chunk[place_col].astype(str).str.contains('india', case=False, na=False)
        
    # Check Brands
    brand_col = next((c for c in df_chunk.columns if 'brand' in c.lower()), None)
    if brand_col:
        mask |= df_chunk[brand_col].astype(str).str.contains('Amul|Britannia|Parle|Tata|Haldiram|Nestle|Dabur|ITC', case=False, na=False)
    
    df_india = df_chunk[mask].copy()

    # C. Process
    if not df_india.empty:
        # 1. Clean Names
        name_col = next((c for c in df_india.columns if 'name' in c.lower() and 'generic' not in c.lower()), 'product_name')
        if name_col in df_india.columns:
             df_india['clean_name'] = df_india[name_col].apply(clean_name)
        else:
             df_india['clean_name'] = "Unknown"

        # 2. Extract Nutrition
        if nutri_col:
            df_india['sugar'] = df_india[nutri_col].apply(lambda x: universal_extractor(x, 'sugars'))
            df_india['fat'] = df_india[nutri_col].apply(lambda x: universal_extractor(x, 'fat')) # or 'saturated-fat'
            df_india['protein'] = df_india[nutri_col].apply(lambda x: universal_extractor(x, 'proteins'))
            df_india['calories'] = df_india[nutri_col].apply(lambda x: universal_extractor(x, 'energy-kcal'))
        else:
            # Emergency fill
            df_india['sugar'] = 0.0; df_india['fat'] = 0.0; df_india['protein'] = 0.0; df_india['calories'] = 0.0

        # Keep relevant columns
        cols_to_keep = ['code', 'clean_name', 'brands', 'nutriscore_grade', 'image_url', 'sugar', 'fat', 'protein', 'calories']
        available_cols = [c for c in cols_to_keep if c in df_india.columns]
        
        # Add the ones we just created manually
        for c in ['clean_name', 'sugar', 'fat', 'protein', 'calories']:
            if c not in available_cols: available_cols.append(c)

        df_final_chunk = df_india[available_cols].copy()
        indian_products.append(df_final_chunk)
        
        print(f"   Batch {batch_count}: Found {len(df_final_chunk)} items.")

    # Stop at 200 batches for a good dataset (or remove limit)
    if batch_count >= 200: break

# --- 4. SAVE ---
if indian_products:
    final_df = pd.concat(indian_products, ignore_index=True)
    
    # Rename for final CSV
    final_df = final_df.rename(columns={'code': 'barcode', 'clean_name': 'name', 'brands': 'brand', 'nutriscore_grade': 'grade'})
    
    # Filter out empty rows (Optional)
    # final_df = final_df[ (final_df['calories'] > 0) | (final_df['sugar'] > 0) ]

    output_path = os.path.join(PROJECT_DIR, "india_products_v2.csv")
    final_df.to_csv(output_path, index=False)
    print(f"\n🎉 DONE! Saved {len(final_df)} products to {output_path}")
    display(final_df.head())
else:
    print("❌ No items found.")

🚀 RE-STARTING FINAL EXTRACTION (Universal Parser)...
   Batch 1: Found 2 items.
   Batch 2: Found 468 items.
   Batch 3: Found 84 items.
   Batch 4: Found 34 items.
   Batch 5: Found 7 items.
   Batch 6: Found 205 items.
   Batch 7: Found 57 items.
   Batch 8: Found 28 items.
   Batch 9: Found 60 items.
   Batch 10: Found 25 items.
   Batch 11: Found 10 items.
   Batch 12: Found 25 items.
   Batch 13: Found 12 items.
   Batch 14: Found 86 items.
   Batch 15: Found 188 items.
   Batch 16: Found 114 items.
   Batch 17: Found 43 items.
   Batch 18: Found 135 items.
   Batch 19: Found 111 items.
   Batch 20: Found 58 items.
   Batch 21: Found 17 items.
   Batch 22: Found 6 items.
   Batch 23: Found 6 items.
   Batch 24: Found 574 items.
   Batch 25: Found 12 items.
   Batch 26: Found 48 items.
   Batch 27: Found 4 items.
   Batch 28: Found 2 items.
   Batch 29: Found 12 items.
   Batch 31: Found 12 items.
   Batch 32: Found 1 items.
   Batch 33: Found 2 items.
   Batch 34: Found 57 items.


,barcode,name,brand,grade,sugar,fat,protein,calories
0,0011433110587,[unknown],deep indian kitchen,b,1.18,6.67,5.10,122.0
1,0011433119696,[pizza-average],Deep Indian Kitchen,b,3.62,6.67,10.50,267.0
2,0012000713019,[unknown],Nestlé,unknown,0.00,0.00,0.00,0.0
3,0013454377024,[sauce-pesto-prepacked],Stonemill Kitchens,e,0.00,166.00,25.50,1650.0
4,0013800143310,[unknown],Nestlé,c,3.72,3.72,6.05,112.0


In [15]:
import pandas as pd
import numpy as np
import os

# --- 1. SETUP ---
PROJECT_DIR = r"C:\Users\arjun\Downloads\Nutri Rate"
# Make sure this matches the exact name of your downloaded file
INDIAN_CSV = r"C:\Users\arjun\Downloads\Indian_Food_Nutrition_Processed.csv"

print("🚀 PROCESSING COOKED INDIAN FOOD...")

# --- 2. DEFINE GRADING ALGORITHM (The "AI" Logic) ---
def calculate_grade(row):
    """
    Calculates a simple A-E grade based on Indian Diet context.
    """
    # 1. Negative Points (Bad stuff)
    points = 0
    
    # Sugar (High penalty)
    if row['sugar'] > 22.5: points += 10
    elif row['sugar'] > 10: points += 6
    elif row['sugar'] > 4.5: points += 2
    
    # Fat (Medium penalty)
    if row['fat'] > 15: points += 5
    elif row['fat'] > 5: points += 2
    
    # Calories (Density penalty)
    if row['calories'] > 300: points += 4
    elif row['calories'] > 150: points += 2
    
    # 2. Positive Points (Good stuff)
    # Protein (Bonus)
    if row['protein'] > 15: points -= 5
    elif row['protein'] > 8: points -= 3
    
    # 3. Final Score Mapping
    if points <= -2: return 'A'
    if points <= 2: return 'B'
    if points <= 8: return 'C'
    if points <= 15: return 'D'
    return 'E'

# --- 3. LOAD & CLEAN ---
try:
    df_dishes = pd.read_csv(INDIAN_CSV)
    print(f"Loaded {len(df_dishes)} raw dishes.")
    
    # Inspect columns to map them correctly
    # Usually these datasets have 'Carbohydrates', 'Total Fat', 'Protein', 'Energy'
    # We rename them to our standard schema
    
    # NOTE: Adjust these keys if your CSV has different names (e.g. "Carbs" vs "Carbohydrates")
    # This mapping is based on the Kaggle dataset you described earlier
    rename_map = {
        'Dish Name': 'name',
        'Calories': 'calories', 
        'Protein (g)': 'protein',
        'Fat (g)': 'fat',
        'Carbohydrates (g)': 'sugar', # Using Carbs as Sugar proxy for cooked food
    }
    
    # If columns are missing, try generic names
    if 'Dish Name' not in df_dishes.columns:
        # Fallback for standard Kaggle "indian_food.csv"
        rename_map = {
            'name': 'name',
            'calories': 'calories',
            'protein': 'protein', 
            'fats': 'fat',
            'carbohydrates': 'sugar'
        }

    df_dishes = df_dishes.rename(columns=rename_map)
    
    # Ensure all numeric columns exist and are numbers
    for col in ['calories', 'protein', 'fat', 'sugar']:
        if col not in df_dishes.columns:
            df_dishes[col] = 0.0
        else:
            # Force convert to number, coerce errors to 0
            df_dishes[col] = pd.to_numeric(df_dishes[col], errors='coerce').fillna(0.0)

    # --- 4. CALCULATE GRADES ---
    print("Calculating Nutri-Scores for dishes...")
    df_dishes['grade'] = df_dishes.apply(calculate_grade, axis=1)
    
    # Add metadata
    df_dishes['brand'] = 'Home/Restaurant'
    df_dishes['barcode'] = 'no_barcode' # Cooked food has no barcode
    df_dishes['source'] = 'IndianDishes'
    
    # Keep only standard columns
    final_cols = ['barcode', 'name', 'brand', 'grade', 'sugar', 'fat', 'protein', 'calories', 'source']
    df_dishes_clean = df_dishes[final_cols].copy()
    
    # --- 5. SAVE ---
    output_path = os.path.join(PROJECT_DIR, "indian_dishes_clean.csv")
    df_dishes_clean.to_csv(output_path, index=False)
    
    print(f"🎉 SUCCESS! Processed {len(df_dishes_clean)} dishes.")
    print(f"Saved to: {output_path}")
    display(df_dishes_clean.head())

except Exception as e:
    print(f"❌ Error: {e}")
    print("Columns found in file:", df_dishes.columns.tolist() if 'df_dishes' in locals() else "None")

🚀 PROCESSING COOKED INDIAN FOOD...
Loaded 1014 raw dishes.
Calculating Nutri-Scores for dishes...
🎉 SUCCESS! Processed 1014 dishes.
Saved to: C:\Users\arjun\Downloads\Nutri Rate\indian_dishes_clean.csv


,barcode,name,brand,grade,sugar,fat,protein,calories,source
0,no_barcode,Hot tea (Garam Chai),Home/Restaurant,B,2.58,0.0,0.39,0.0,IndianDishes
1,no_barcode,Instant coffee,Home/Restaurant,B,3.65,0.0,0.64,0.0,IndianDishes
2,no_barcode,Espreso coffee,Home/Restaurant,B,6.62,0.0,1.75,0.0,IndianDishes
3,no_barcode,Iced tea,Home/Restaurant,B,2.70,0.0,0.03,0.0,IndianDishes
4,no_barcode,Raw mango drink (Aam panna),Home/Restaurant,B,9.05,0.0,0.16,0.0,IndianDishes


In [16]:
print("🔄 MERGING EVERYTHING...")

# 1. Load Packaged Food
packaged_file = os.path.join(PROJECT_DIR, "india_products_v2.csv")
df_packaged = pd.read_csv(packaged_file)
df_packaged['source'] = 'OpenFoodFacts'

# 2. Load Cooked Food
cooked_file = os.path.join(PROJECT_DIR, "indian_dishes_clean.csv")
if os.path.exists(cooked_file):
    df_cooked = pd.read_csv(cooked_file)
else:
    df_cooked = pd.DataFrame()
    print("⚠️ Warning: Cooked food file not found. Skipping.")

# 3. Combine
master_df = pd.concat([df_packaged, df_cooked], ignore_index=True)

# 4. Final Polish (Fill NaNs with 0 or Unknown)
master_df['name'] = master_df['name'].fillna("Unknown Product")
master_df['brand'] = master_df['brand'].fillna("Unknown Brand")
master_df[['sugar', 'fat', 'protein', 'calories']] = master_df[['sugar', 'fat', 'protein', 'calories']].fillna(0.0)

# 5. Save the API Database
master_path = os.path.join(PROJECT_DIR, "master_food_database.csv")
master_df.to_csv(master_path, index=False)

print(f"🏆 CHAMPION! Master Database Created with {len(master_df)} items.")
print(f"📂 Location: {master_path}")
print("\nSample Data:")
display(master_df.sample(5))

🔄 MERGING EVERYTHING...
🏆 CHAMPION! Master Database Created with 15440 items.
📂 Location: C:\Users\arjun\Downloads\Nutri Rate\master_food_database.csv

Sample Data:


,barcode,name,brand,grade,sugar,fat,protein,calories,source
9785,42272905355,['soup-average'],Amy's Kitchen Inc.,b,2.010000,1.76,2.76,67.800003,OpenFoodFacts
15258,no_barcode,Lotus stem pickle (Kamal kakdi ka achar),Home/Restaurant,C,20.070000,0.00,1.44,0.000000,IndianDishes
2033,3023290220689,['unknown'],Nestlé,d,19.500000,5.90,4.70,158.000000,OpenFoodFacts
9281,7613035357549,['unknown'],Nestlé,unknown,57.500000,31.10,6.30,542.000000,OpenFoodFacts
12595,4600680014982,Unknown Product,Nestle,e,48.900002,27.60,5.70,521.000000,OpenFoodFacts


In [17]:
import pandas as pd
import os

# CONFIG
CSV_PATH = r"C:\Users\arjun\Downloads\Nutri Rate\master_food_database.csv"

print("🔍 INSPECTING BARCODE: 978542272905355")

# 1. Load the dataframe WITHOUT any type conversion first
df = pd.read_csv(CSV_PATH, dtype=object) 

# 2. Search for the row using loose matching
# We look for any row where the barcode *contains* "978542" (first 6 digits)
# This helps us find it even if the end is cut off or formatted weirdly.
mask = df['barcode'].astype(str).str.contains("978542", na=False)
results = df[mask]

if not results.empty:
    print(f"\n✅ Found {len(results)} potential matches.")
    print("Here is exactly how they look in the CSV:")
    
    for idx, row in results.iterrows():
        raw_val = row['barcode']
        name = row['name']
        print(f"\n--- Row {idx} ---")
        print(f"Product: {name}")
        print(f"RAW BARCODE VALUE: '{raw_val}'")
        print(f"Type: {type(raw_val)}")
        
        # Test if it equals the target
        target = "978542272905355"
        print(f"Match Check: '{raw_val}' == '{target}' ? {str(raw_val) == target}")
else:
    print("\n❌ CRITICAL: The barcode 978542... is NOT in the database at all.")
    print("Let's check the first 5 barcodes in the file to see the format:")
    print(df['barcode'].head().tolist())

🔍 INSPECTING BARCODE: 978542272905355

❌ CRITICAL: The barcode 978542... is NOT in the database at all.
Let's check the first 5 barcodes in the file to see the format:
['11433110587', '11433119696', '12000713019', '13454377024', '13800143310']
